In [2]:
import ee
ee.Initialize()
from iteru import *
from ipywidgets import *


In [3]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [6]:
draw_control.last_draw

NameError: name 'draw_control' is not defined